# Interactive molecular dynamics

So far you have seen how to use BioSimSpace to write workflow components and run them in a Jupyter notebook, or from the command-line. BioSimSpace is also a great tool for playing around with molecular simulations directly and interacting with them in real-time. In this notebook you'll learn how to use BioSimSpace to set up and run an equilibration protocol, then query the running process for information, plot graphs of the latest data, visualise molecular configurations, and analyse trajectory data.

Before we get started, let's import BioSimSpace so that it's available inside of our notebook.

In [ ]:
import BioSimSpace as BSS

## Creating a molecular system

First of all we need to load a molecular system. Once again, we'll use the examples from the `input` directory.

In [ ]:
system = BSS.IO.readMolecules(["input/ala.crd", "input/ala.top"])

We have now created a molecular system. The system consists of an alanine dipeptide molecule in a box of water. To show the number of molecules in the system, run:

In [ ]:
system.nMolecules()

## Defining a simulation protocol

BioSimSpace provides functionality for defining various simulation protocols. In this notebook we will construct a typical simulation workflow that uses a sequence of simple protocols, with the output of one forming the input of the next:

1. _Minimisation:_ Energy minimisation the molecular system.
2. _Equilibration:_ Equlibration of the system to a target temperature.
3. _Production:_ Regular molecular dynamics, run at fixed temperature.
4. _Custom_: A user defined protocol, e.g. a config file for a molecular dynamics package.

When defining a protocol we are configuring the type of simulation that we wish to run, as well as any options for the particular simulation. For example, to create a default equilibration protocol:

```python
protocol = BSS.Protcol.Equilibration()
```

This defines a 0.2 nanosecond equilibration protocol at a temperature of 300 Kelvin. For convenience, let's reduce the runtime. We'll also slowly heat the system during equilibration and restrain the position of atoms in the protein backbone.

In [ ]:
# Initialise a short equilibration protocol.
protocol = BSS.Protocol.Equilibration(runtime=0.05*BSS.Units.Time.nanosecond,
                                      temperature_start=0*BSS.Units.Temperature.kelvin,
                                      temperature_end=300*BSS.Units.Temperature.kelvin,
                                      restrain_backbone=True)

## Initialising a process

We now have everything that is needed to create a process object. To do so, run:

In [ ]:
process = BSS.MD.run(system, protocol, autostart=False)

On creation, BioSimSpace searches your `PATH` for an appropriate executable for running the process. The executable that is chosen may be dependent on the available hardware and type of protocol.

To see the executable that was chosen, run:

In [ ]:
process.exe()

By default, BioSimSpace runs each process inside a unique temporary workspace. This is where all of the input and configuration files will be created, as well as any of the output when the process is run.

To view the working directory, run:

In [ ]:
process.workDir()

To get a list of the autogenerated input files:

In [ ]:
process.inputFiles()

## Configuring the process

For each protocol, BioSimSpace will initialise default configuration parameters appropriate to the process based on best practice in the field.

To see the list of configuration parameter strings, run:

In [ ]:
process.getConfig()

In some cases, it may be desirable to run a custom protocol. BioSimSpace provides several ways of acheiving this:

* By setting the configuration of an existing process:

```python
# Set the configuration from file.
process.setConfig("my_config.txt")

# Set the configuration using a list of configuration strings.
my_config = ["some config parameter string", "another config parameter string"]
process.setConfig(my_config)
```

* By adding to the configuration of an existing process:

```python
# Add using a configuration from file.
process.addToConfig("my_config.txt")

# Add a list of parameter strings to the configuration.
my_config = ["some config parameter string", "another config parameter string"]
process.addToConfig(my_config)
```

## Configuring command-line arguments

Where necessary, BioSimSpace will configure the command-line arguments needed to run the process.

To view the command-line argument string, run:

In [ ]:
process.getArgString()

The arguments are stored internally as an `OrderedDict` object. To view it, run:

In [ ]:
process.getArgs()

BioSimSpace provides functionality for setting and manipulating the arguments. For example, to disable the overwriting of output files:

In [ ]:
process.setArg('-O', False)
process.getArgs()

Let's see how the argument string changed:

In [ ]:
process.getArgString()

The `setArg` method can be used to add a new argument, or to overwrite the value of an existing argument. There are several other methods that allow the arguments to be modified:

* setArgs(args): Overwrite all arguments with a new dictionary.
* addArgs(args): Append additional arguments.
* insertArgs(arg, value, index): Insert an argument at a specific index.
* deleteArg(arg): Delete an argument from the dictionary.
* clearArgs(): Clear all of the arguments.

If you ever get in trouble, it's easy to reset the arguments to their default values:

In [ ]:
process.resetArgs()
process.getArgs()

## Running the process

Having configured the process to your liking, it's time to run a simulation.

To start the process, run:

In [ ]:
process.start()

BioSimSpace has now launched a minimisation process in the background. (Note that if we have omitted the `autostart` keyword argument when initialising the process object then it would have been immediatedly started.)

To see if the process is still running:

In [ ]:
process.isRunning()

To see how many minutes the process has been running for:

In [ ]:
process.runTime()

We can also query the total energy of the molecular system (in kcal/mol):

In [ ]:
process.getTotalEnergy()

We can monitor the time, temperature, and energy as the process runs. If you run this multiple times using "CTRL+Return" you'll see the temperature slowly increasing.

In [ ]:
print(process.getTime(), process.getTemperature(), process.getTotalEnergy())

It's possible to query many other thermodynamic records. What's available depends on type of protocol and the program that is used to run the protocol. To get more information, run:

In [ ]:
help(process)

## Plotting time series data

As well as querying the most recent records we can also get a time series of results by passing the `time_series` keyword argument to any of the data record getter methods, e.g.

```python
# Get a time series of pressure records.
pressure = process.getPressure(time_series=True)
```

BioSimSpace comes with several useful tools that are available when working inside of a Jupyter notebook. One of this is the `plot` function, that allows us to create simple x/y plot of time series data.

Let's grab the same record data as above and use it to make some graphs of the data.

In [ ]:
# Generate a plot of time vs temperature.
plot1 = BSS.Notebook.plot(process.getTime(time_series=True), process.getTemperature(time_series=True))

# Generate a plot of time vs energy.
plot2 = BSS.Notebook.plot(process.getTime(time_series=True), process.getTotalEnergy(time_series=True))

Re-run the cell using "CTRL+Return" to see the graphs update as the simulation progesses.

Being able to query a process in real time is an incredibly useful tool. This could enable us to check for convergence, or spot errors in the simulation. If you ever need to kill a running process (perhaps it was configured incorrectly), run:

```python
process.kill()
```

## Visualising the molecular system

Another useful tool that is available when working inside of a notebook. One of these is the `View` class that can be used to visualise the molecular system while a process is running. To create a `View` object we must attach it to a process (or a molecular system), e.g.:

In [ ]:
view = BSS.Notebook.View(process)

We can now visualise the system:

In [ ]:
view.system()

To only view a specific molecule:

In [ ]:
view.molecule(0)

To view a list of molecules:

In [ ]:
view.molecules([0, 5, 10])

If a particular view was of interest it can be reloaded as follows:

In [ ]:
# Reload the original view.
view.reload(0)

To save a specific view as a PDB file:

In [ ]:
view.savePDB("my_view.pdb", index=0)

## Reading and analysis trajectory data

BioSimSpace comes with a set of tools for reading and analysis trajectory files. Files can be loaded directly, or if supported, can be read from a running process.

For example, to get the trajectory from the process, run:

In [ ]:
traj = process.getTrajectory()

(If you get an error, then the trajectory file may be in the process of being written. Simply try again.)

To get the current number of frames:

In [ ]:
traj.nFrames()

To get the frames as a list of system objects:

In [ ]:
frames = traj.getFrames()

Specific frames can be extracted by passing a list of indices, e.g. the first and last:

In [ ]:
frames = traj.getFrames([0, -1])

Like most things in BioSimSpace, the `Trajectory` class is simply a wrapper around existing tools. Internally, trajectories are stored as an [MDTraj](http://mdtraj.org) object. This can be obtained, allowing the user direct access to the full power of MDTraj.

In [ ]:
mdtraj = traj.getTrajectory()
type(mdtraj)

Alternatively, a trajectory can be returned in [MDAnalysis](https://www.mdanalysis.org) format:

In [ ]:
mdanalysis = traj.getTrajectory(format="mdanalysis")
type(mdanalysis)

The `Trajectory` class provides wrappers around some basic MDTraj analysis tools, allowing the user to compute quantities such as the root mean squared displacement (RMSD).

Let's measure the RMSD of the alanine-dipeptide molecule with a reference to its configuration in the first trajectory frame. To extract the alanine-dipeptide, we search the system for a residue named `ALA`. We'll also plot the RMSD for each frame of the trajectory.

In [ ]:
BSS.Notebook.plot(y=process.getTrajectory().RMSD(frame=0, molecule=system.getMolWithResName("ALA")), xlabel="Frame", ylabel="RMSD")

Finally, let's save the equilibrated system along with a file containing the RMSD for each trajectory frame. Once again we'll use `block=True` to indicate that we want to wait until the process has finished before writing to file.

In [ ]:
# Write the final configuration of the equilibration to file.
BSS.IO.saveMolecules("equilibrated", process.getSystem(block=True), system.fileFormat())

# Get the RMSD and write it to file. There is no need to block here since we know the process has already finished.
rmsd = process.getTrajectory().RMSD(frame=0, molecule=system.getMolWithResName("ALA"))

with open("rmsd.txt", "w") as file:
    for index, value in enumerate(rmsd):
        file.write("%d %5.4f\n" % (index, value))

## Exercise

Open a new notebook and create a BioSimSpace node to execute the equilibration protocol outlined above.

The node should require the following input:

* A set of input files defining the molecular system.
* The runtime, in nanoseconds.
* The starting temperature, in Kelvin.
* The final temperature, in Kelvin.
* Whether to restrain the backbone.
* The name of the residue used to identify the molecule.

More details regarding the available input requirements can be found [here](https://github.com/michellab/BioSimSpace/tree/devel/python/BioSimSpace/Gateway).

The node should create the following ouput:

* The equilibrated molecular system, saved in the original file format.
* The RMSD, saved as a text file.

When you are happy with your node, save it as a python script and verify that it works from the command-line.

If you get stuck, a solution can be found [here](nodes/equilibration.ipynb).